# Introduction to the `sqlite3` library

**`SQLite`** is a C library that provides a lightweight disk-based database that doesn’t require a separate server process. Applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then later port the application to a production database system.   

`SQLite` databases are stored in a file on disk (usually with a `.db` extension). If you attempt to connect to a database file that doesn't exist, `SQLite` with create a new database, assign it the name you passed to the connect function and save it to your current working directory. 

Typical sqlite setup and usage in Python is as follows:

*  **Create a connection object** - For `SQLite`, pass the database filepath instead of authentication details:

            db = sqlite3.connect(<filename>.db)
            
*  **Define a cursor** - Call the `cursor()` method on the connection object created above:

            cursor = db.cursor()  
            
*  **Construct a query for the dataset of interest** - This can be almost any valid SQL command:

            SQL = "SELECT * FROM SAMPLE_TABLE"  
            
*  **Pass the query string to the cursor's `execute` method:**

            cursor.execute(SQL) 
            
            
*  **Iterate over the cursor** - Using a `for` loop, list comprehensions, etc...  

  
The result will be a list of tuples, so data elements can be accessed by row or selectively by referencing components by index offset: 

    for record in cursor: print(record)

**Creating Databases and Tables**
--------------------

If the database file passed to the `sqlite3.connect` method doesn't exist, a new database with the name specified will be created. The following example creates a database consisting of 2 tables: The first table holds closing stock prices, the second contains a mapping between ticker symbols and company names:

NOTE: For more information on `SQLite` datatypes and the resulting affinity mappings of common datatypes for   
other RDBMS, follow this link and navigate to the *Affinity Name Examples* section:  

https://www.sqlite.org/datatype3.html

In [ ]:
"""
=====================================
Toyota sales table                   |
=====================================
Table name: `TOYOTA_SALES`          |

Fields: 
      price           REAL          |
      age             INTEGER       |
      mileage         INTEGER       |
      fuel_type       TEXT          |
      horse_power     INTEGER       |
      metalic_color   INTEGER       |
      automatic       INTEGER       |
      CC              INTEGER       |
      doors           INTEGER       |
      quarterly_tax   INTEGER       |
      weight          INTEGER       |
------------------------------------

First two rows from toyota_sales.csv:

price, age, mileage, fuel_Type, horse_Power, metalic_color,  automatic, CC,   doors, quarterly_tax, weight
13500, 23,  46986,   Diesel,    90,          1,              0,         2000, 3,     210,           1165
           
"""
import sqlite3

# Create new database `sales.db`. Notice `sales.db` is now 
# listed in your working directory:
db = sqlite3.connect("sales.db")

# Initiate a cursor, and call the connection's cursor method:
cursor = db.cursor()

# Table structures for toyta sale
sales_table = """CREATE TABLE TOYOTA_SALES (
            price           REAL,          
            age             INTEGER,       
            mileage         INTEGER,       
            fuel_type       TEXT,          
            horse_power     INTEGER,       
            metalic_color   INTEGER,       
            automatic       INTEGER,       
            CC              INTEGER,       
            doors           INTEGER,       
            quarterly_tax   INTEGER,       
            weight          INTEGER )"""

# Call the `cursor.execute` method, passing sales_table as argument:
cursor.execute(sales_table)

# IMPORTANT! Be sure to commit changes you want to persist.
# Without commiting, changes will not be saved:
db.commit()

# close connection to `sales.db`:
db.close()

To verify that your tables have been created, run the following:

In [ ]:
# Re-connect to `sales.db`:
db     = sqlite3.connect('sales.db')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())
db.close()

**Loading Data into `SQLite` Database Tables**
--

The following example demonstrates two methods of loading data into `SQLite` tables. The first method assumes the data is already available in memory in the current Python session. The second method assumes data is being loaded from a .csv file:

####  Method #1: Data already avialable in Python session 

In [ ]:
# Reestablish connection to `sales.db` database:
db     = sqlite3.connect('sales.db')
cursor = db.cursor()

# Insert two records into `TOYOTA_SALES` table
#9900,64,41200,Petrol,110,0,0,1600,5,85,1070
#10950,57,40214,Petrol,86,0,0,1300,3,69,1025


# Single records can be inserted using the `cursor.execute` method:
cursor.execute("INSERT INTO TOYOTA_SALES VALUES (9900, 64, 41200, 'Petrol', 110, 0, 0, 1600, 5, 85, 1070)")
cursor.execute("INSERT INTO TOYOTA_SALES VALUES (10950, 57, 40214, 'Petrol', 86, 0, 0, 1300, 3, 69, 1025)")

# Not forgetting to commit changes and close connection:
db.commit()
db.close()


We can also insert multiple records at once using lists

In [ ]:
# Reestablish connection to `sales.db` database:
db     = sqlite3.connect('sales.db')
cursor = db.cursor()

# We can insert several records at once if we create a list 
# of tuples of the data to insert, then call `cursor.executemany`:
sales_records = [(12950, 23, 71138, 'Diesel', 69, 0, 0, 1900, 3, 185, 1105),
               (9950, 63, 41586, 'Petrol', 110, 1, 0, 1600, 5, 19, 1114),
               (9950, 67, 42102, 'Petrol', 110, 1, 0, 1600, 5, 85, 1075)]

# Calling `cursor.executemany`
cursor.executemany("INSERT INTO TOYOTA_SALES VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", sales_records)

# Not forgetting to commit changes and close connection:
db.commit()
db.close()

The `(?,?,?)` in `cursor.executemany` serve as placeholders for columns in the target table. There should be one `?` for each column in the target table. The more common scenario may be loading data from file into an `SQLite` database table. The syntax is similiar, with added file handling logic:

####  Method #1: Data read in from .csv 

In [ ]:
# Requires `ticker_data.csv` file
import sqlite3
import csv

# Reestablish connection to the database:
db     = sqlite3.connect('sales.db')
cursor = db.cursor()

# Open `ticker_data.csv`, and create a csv.reader instance:
with open('toyota_sales.csv', 'r') as f:  # on Windows, change `r` to `rb`
    
    fcsv = csv.reader(f)
    
    # `recs_to_load` is a list of records contained in `ticker_data.csv`:
    recs_to_load = [record for record in fcsv]

    # call `cursor.executemany`, specifying `recs_to_load`
    cursor.executemany("INSERT INTO TOYOTA_SALES VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", recs_to_load)

# Not forgetting to commit changes and close connection:
db.commit()
db.close()

**`SQLite` Data Retrieval**
==================

Retrieval of `SQLite` database records is carried out in exactly the same as with `cx_Oracle`. An iterator in the form  
of a database cursor is returned, which is traversed, yielding the query is question's result set:

In [ ]:
import sqlite3

# reestablish connection to `sales.db` database:
db     = sqlite3.connect('sales.db')
cursor = db.cursor()

#construct a query to retrieve data from `TOYOTA_SALES`:
SQL = "SELECT * FROM TOYOTA_SALES"

# call `cursor.execute` on query string:
cursor.execute(SQL)

# `cursor` can be iterated over:
for rec in cursor:
    print(rec)
    
# not forgetting to commit changes and close connection:   
db.commit()
db.close()


Headers need to be extracted from the `cursor.description` attribute:

In [ ]:
# Obtain reference to table headers:
import sqlite3

# Reestablish connection to `sales.db` database:
db     = sqlite3.connect('sales.db')
cursor = db.cursor()

#construct a query to retrieve data from `TOYOTA_SALES`:
SQL = "SELECT * FROM TOYOTA_SALES"

# Call `cursor.execute` on SQL:
cursor.execute(SQL)

# Capture table headers into `headers` list:
headers = [i[0] for i in cursor.description]
   
# Not forgetting to commit changes and close connection:   
db.commit()
db.close()

print("TOYOTA_SALES table fieldnames: {}".format(headers))



**Useful Links:**
---------------

*  Python `sqlite3` library documentation: https://docs.python.org/3/library/sqlite3.html#module-sqlite3   
*  `SQLite` Documentation: https://www.sqlite.org/docs.html   
*  Well-known users of `SQLite`: https://www.sqlite.org/famous.html  

# Exercise 1

1. Create a database named "*stocks*" 
2. Create a table called "*stocks-500*" using the same column names and structures used in the "*Stocks500.csv*" file.
3. Import the "*Stocks500.csv*" file into the "*stocks-500*" table.

# Exercise 2

1. Create a database named "*funds*" 
2. Create a table called "*mutual-funds*" using the same column names and structures used in the "*MutualFunds.csv*" file.
3. Import the "*MutualFunds.csv*" file into the "*mutual-funds*" table.